In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoModelForMaskedLM,XLMRobertaForSequenceClassification

In [ ]:
model_name = '' #trasnformer model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base",num_labels=5)
# Define device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import pandas as pd
df_url="https://drive.google.com/file/d/1ekqMGq-Mnlqsh1XC2ykJ9zCzUo78dnRK/view?usp=sharing"
df_datapath='https://drive.google.com/uc?export=download&id='+df_url.split('/')[-2]
df=pd.read_csv(df_datapath)
stop_words_df = pd.read_excel('/kaggle/input/bangla-stopwords/stopwords_bangla.xlsx',index_col=False)

In [5]:
from sklearn.model_selection import train_test_split
df_train,df_test= train_test_split(df, test_size=0.33, random_state=42, shuffle=True)

In [6]:
df_train.head()

,articleID,domain,date,category,source,relation,text,content,label,F-type
4228,3520,banglanews24.com,2018-09-19 19:18:24,Finance,"অর্থমন্ত্রী,পিআরআই’র চেয়ারম্যান",Related,দক্ষিণ এশিয়ায় বাংলাদেশের বাণিজ্য দ্বিগুণ করা স...,ঢাকা: দক্ষিণ এশিয়ায় বাংলাদেশের বাণিজ্য হচ্ছে ৭...,1,NaN
799,3911,channelionline.com,2018-09-20 16:15:08,Lifestyle,কানাডার বিএনএন ব্লুমবার্গ টিভি চ্যানেল,Related,গাঁজার স্বাদযুক্ত কোমল পানীয় আনতে আগ্রহী কোকা...,গাঁজা থেকে নতুন স্বাদের পানীয় উৎপাদন করার কথা ...,1,NaN
2402,963,earki.com,"২১:৩১, ফেব্রুয়ারি ০৬, ২০১৯",Miscellaneous,Reporter,Unrelated,পদবীতে খালিসিকে পেছনে ফেলে দিলেন মাহফুজুর রহমা...,গত ৪ ফেব্রুয়ারি ‘সময় ও অসময়ের গল্প’ সিরিজের না...,0,Satire
5187,1512,dailynayadiganta.com,2018-09-20 23:14:52,Sports,Reporter,Related,শুক্রবারও বড় জয় চায় মারিয়ারা [ SEP ] প্রথম দুই...,প্রথম দুই খেলায় ১৮ গোলের জয় বাংলাদেশের। অথচ এই...,1,NaN
1073,6183,dailynayadiganta.com,2018-09-20 14:50:26,Miscellaneous,বিএফইউজ,Related,ডিজিটাল নিরাপত্তা আইনে স্বাক্ষর না করার জন্য র...,"সাংবাদিক সমাজ, সম্পাদক পরিষদ, নাগরিক সমাজ ও দে...",1,NaN


In [7]:
df_train = df_train.rename({'text': 'text', 'label': 'label'}, axis=1)
df_test = df_test.rename({'text': 'text', 'label': 'label'}, axis=1)

In [8]:
STOPWORDS = set([word.strip() for word in stop_words_df['words']])

In [9]:
import re
def preprocess(x):
    html_pattern = re.compile('<.*?>')
    x = html_pattern.sub(r'', x)
    x = " ".join([word for word in str(x).split() if word not in STOPWORDS])
    return x
df_train['text'] = df_train['text'].apply(lambda x: preprocess(x))
df_test['text'] = df_test['text'].apply(lambda x:preprocess(x))

In [10]:
import numpy as np
allcats = set(df_train['label'].dropna().tolist())
# allcats.add('Correct')
labeldict = {}
counter = 0
for i in allcats:
    labeldict[i] = counter
    counter += 1
labeldict

{0: 0, 1: 1}

In [11]:
def manage(x):
    if x in labeldict:
        return labeldict[x]
   
df_train['label'] = df_train['label'].apply(lambda x:manage(x))
df_test['label'] = df_test['label'].apply(lambda x:manage(x))

In [12]:
data_no = 5

# Prepare the training data
train_texts = df_train['text'].tolist()
train_labels = df_train['label'].tolist()

test_texts = df_test['text'].tolist()
test_labels = df_test['label'].tolist()

In [13]:
# Tokenize and encode the training texts
train_encodings = tokenizer(train_texts, truncation=True, max_length=128,
        padding='max_length', return_tensors = 'pt')

# Convert the labels to tensors
train_labels = torch.tensor(train_labels)

# Create a PyTorch dataset
train_dataset = torch.utils.data.TensorDataset(train_encodings['input_ids'],
                                               train_encodings['attention_mask'],
                                               train_labels)

# Create a data loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

model = model.to(device)

In [14]:
from tqdm.notebook import tqdm
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import accuracy_score

# Set the model to training mode
model.train()

# Define the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

losses = []
accuracies = []  # To store accuracy per epoch
num_epochs = 10
# Training loop
for epoch in tqdm(range(num_epochs)):  # Number of training epochs
    running_loss = 0.0
    predicted_labels = []  # To store predicted labels for accuracy calculation
    true_labels = []  # To store true labels for accuracy calculation

    for batch in tqdm(train_loader):
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Convert logits to predicted labels
        _, predicted = torch.max(logits, dim=1)
        predicted_labels.extend(predicted.cpu().tolist())
        true_labels.extend(labels.cpu().tolist())

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)

    # Calculate and store accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)
    accuracies.append(accuracy)

    print(f'Epoch {epoch + 1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {accuracy:.4f}')

# Save the model
torch.save(model.state_dict(), 'model.pth')


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/356 [00:00<?, ?it/s]

Epoch 1/10 - Loss: 0.3515 - Accuracy: 0.8971


  0%|          | 0/356 [00:00<?, ?it/s]

Epoch 2/10 - Loss: 0.1258 - Accuracy: 0.9607


...

Epoch 8/10 - Loss: 0.0233 - Accuracy: 0.9939


  0%|          | 0/356 [00:00<?, ?it/s]

Epoch 9/10 - Loss: 0.0234 - Accuracy: 0.9935


  0%|          | 0/356 [00:00<?, ?it/s]

Epoch 10/10 - Loss: 0.0139 - Accuracy: 0.9967


In [16]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

def predict_labels(text):
    train_encodings = tokenizer(text, truncation=True, max_length=128,
        padding='max_length', return_tensors = 'pt')
    input_ids = train_encodings['input_ids'].to(device)
    attention_mask = train_encodings['attention_mask'].to(device)

    # Set the model to evaluation mode
    model.eval()

    # Disable gradient calculation
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1)
    

    return predicted_class.item(), probabilities[:,1].item()

In [17]:
predicted_labels = []
predicted_probs = []
for text in tqdm(test_texts):
    predicted_label, prob = predict_labels(text)
    predicted_labels.append(predicted_label)
    predicted_probs.append(prob)

# Calculate accuracy and F1 score
accuracy = accuracy_score(test_labels, predicted_labels)
# f1 = f1_score(test_labels, predicted_labels)
# roc_auc = roc_auc_score(test_labels, predicted_probs)

print('Accuracy:', accuracy)
# print('F1 Score:', f1)
# print('ROC-AUC:', roc_auc)

  0%|          | 0/2806 [00:00<?, ?it/s]

Accuracy: 0.9764789736279401


In [18]:
print('Accuracy:', accuracy)

Accuracy: 0.9764789736279401


In [19]:
from sklearn.metrics import roc_auc_score, classification_report

print('\nThe Classification Report is as follows\n')
print(classification_report(test_labels, predicted_labels, digits = 4))


The Classification Report is as follows

              precision    recall  f1-score   support

           0     0.9885    0.8477    0.9127       407
           1     0.9748    0.9983    0.9864      2399

    accuracy                         0.9765      2806
   macro avg     0.9817    0.9230    0.9496      2806
weighted avg     0.9768    0.9765    0.9757      2806

